# Processing the NFT Trait Value Occurrences into JSON Files

This notebook describes the process for creating the NFT trait value occurrences, which is used in `calculations.ipynb` to calculate for trait value scores and the NFT rarity scores and rank. If you are missing the trait value occurence JSON, run this file. However, if you dont have the NFT Round 1 and 2 JSON files as well, please run `webscraper.ipynb` first.

---

### 1.1 Open Statemetnts

In [1]:
open System
open System.Collections.Generic
open System.IO
open System.Text.Json
open System.Reflection

### 1.2 Creating the Type for JSON Serialization
`NFTTraitData` represents the traits of each NFT.

In [2]:
type NFTTraitData = {
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

### 1.3 Calculate the Trait Value Occurences
This function `countAttributes` deserializes the JSON in a given file and then calculates the number of occurences of each trait value over the total number of NFTs. It then returns a Dictionary<string, Dictionary<string, int>> of traits to trait values and their number of occurrences.

In [3]:
let countAttributes (jsonFilePath: string) =

    let jsonString = File.ReadAllText(jsonFilePath)
    let nftList = JsonSerializer.Deserialize<NFTTraitData list>(jsonString)
    let attributeCounts = Dictionary<string, Dictionary<string, int>>()

    let nftAttributes = 
        typeof<NFTTraitData>.GetProperties()
        |> Array.filter (fun prop -> prop.PropertyType = typeof<string>)

    for attribute in nftAttributes do
        let propertyName = attribute.Name
        for nft in nftList do
            let value = attribute.GetValue(nft) :?> string
            if not (isNull value) then
                if not (attributeCounts.ContainsKey(propertyName)) then
                    attributeCounts.[propertyName] <- Dictionary<string, int>()

                let propertyDict = attributeCounts.[propertyName]
                if not (propertyDict.ContainsKey(value)) then
                    propertyDict.[value] <- 0

                propertyDict.[value] <- propertyDict.[value] + 1

    attributeCounts

### 1.4 Serialize and Save Results
The function `serializeAttributes` takes in the output of `countAttributes`, serializes the data, and then saves it in the indicated file.

In [4]:
let serializeAttributes (attributeCounts: Dictionary<string, Dictionary<string, int>>) (outputFilePath: string) =
    try
        let directory = Path.GetDirectoryName(outputFilePath)
        if not (Directory.Exists(directory)) then
            Directory.CreateDirectory(directory) |> ignore

        let jsonOutput = JsonSerializer.Serialize(attributeCounts, JsonSerializerOptions(WriteIndented = true))
        File.WriteAllText(outputFilePath, jsonOutput)
        printfn "Attribute counts saved to %s" outputFilePath
    with
    | ex ->
        printfn "Error during serialization: %s" ex.Message
        raise ex


### 1.5 Call the Functions for Each of the NFT Rounds

In [ ]:
let jsonFilePath = "../data/collection_1.json"
let outputPath = "../data/collectiontraits_1.json"
let attributeCounts = countAttributes jsonFilePath
serializeAttributes attributeCounts  outputPath

Attribute counts saved to ../data/NFTR1Traits.json


In [ ]:
let jsonFilePath = "../data/collection_2.json"
let outputPath = "../data/collectiontraits_2.json"
let attributeCounts = countAttributes jsonFilePath
serializeAttributes attributeCounts outputPath

Attribute counts saved to ../data/NFTR2Traits.json
